# Overview

We now clean more involved features. Due to the data set being relatively small we are able to manually preprerocess given features. In particular:

- location
- details and tags
- crew
- pager code

# Data

In [ ]:
import pandas as pd 

In [ ]:
df = pd.read_csv("..\data\cleaned.csv")
# df.columns
df.head()

In [ ]:
# Filter the DataFrame to get rows where 'pager_code' 
value="333"
filtered_df = df[df['pager_code'] == value]
pd.set_option('display.max_colwidth', None)
filtered_df[['pager_code', 'shout_details']].head(3)

# Location

Based on misspellings of locations and vague descriptions (e.g., "south of Ardlui"), if the location is "south of Ardlui" we record the location "Ardlui"  and in adjustment_km we record south,2km - we take 2km as a rough estimate of the distance south. We input this as (s,2).

We manually input the longitude and latitude coordinates with google maps (DD format).We input the coordinates of the location (Ardlui) into Google Maps and record the core latitude and longitude from there (core_long_lat). It's important to note that the specific location may not be precise during search operations.

In [ ]:
# import pandas as pd 

# df = pd.read_csv("..\data\codes_preprocessed_dates_time_data.csv")
# # df.drop(columns=["shout_details"], inplace=True)
# # df.columns
# df.fillna({"location_of_shout": "Na"}, inplace=True)
feats=['location_of_shout']
df_loc=df[feats]
# df_loc.value_counts()

## Adjustments to given more accurate positions





Adjustments to coordinates 
- Given core_long_lat use adjustment_km to get adj_long_lat

Notes:
  - How to change the coordinates from a given position to say 5km north of it.
  - Recall we working in UK so norther hemisphere (for latitude based around the equator). With respect tothe longitude (based around the prime meridian) with loch lomond positioned in western hemisphere. Lets assume the earth is shaped like a sphere.
  - 1 degree of latitude is 111km

Resources:
  - https://gis.stackexchange.com/questions/5821/calculating-latitude-longitude-x-miles-from-point

- Use adj_long_lat and put onto map 

In [ ]:
if none adjustment copy core to adj_long_lat 

then split adj_long_lat into long and lat

In [ ]:
import math

if adjustment "None" skip
else:


def adjust_coordinates(original_lat, original_lon, distance_km, direction):
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert latitude and longitude to radians
    lat_rad = math.radians(original_lat)
    lon_rad = math.radians(original_lon)
    
    # Calculate change in latitude and longitude based on direction
    if direction == "north":
        delta_lat_deg = distance_km / R * (180.0 / math.pi)
        delta_lon_deg = 0
    elif direction == "south":
        delta_lat_deg = -distance_km / R * (180.0 / math.pi)
        delta_lon_deg = 0
    elif direction == "east":
        delta_lat_deg = 0
        delta_lon_deg = distance_km / (R * math.cos(lat_rad)) * (180.0 / math.pi)
    elif direction == "west":
        delta_lat_deg = 0
        delta_lon_deg = -distance_km / (R * math.cos(lat_rad)) * (180.0 / math.pi)
    else:
        raise ValueError("Invalid direction. Direction must be 'north', 'south', 'east', or 'west'.")
    
    # Calculate new latitude and longitude
    new_lat = original_lat + delta_lat_deg
    new_lon = original_lon + delta_lon_deg
    
    return new_lat, new_lon


In [ ]:

# Example usage:
original_lat = 40.7128  # Example original latitude
original_lon = -74.0060  # Example original longitude
distance_km = 5  # Distance to move
direction = "north"  # Direction to move

new_lat, new_lon = adjust_coordinates(original_lat, original_lon, distance_km, direction)
print("Adjusted Coordinates:", new_lat, "N,", new_lon, "W")


## Automation of position

To-do:

    -[ ] Utilize geopy to obtain the coordinates of the location based on the shout's location (so we can automate the process).
    -[ ] Implement functionality to accept What3Words coordinates.


Provide a function that takes a location and provides core_long_lat value.

Given a map and location data need to be able to convert to longitude/latitude data.

https://geopy.readthedocs.io/en/stable/index.html?highlight=what%20three%20words#geopy.geocoders.What3WordsV3.geocode


# Details and tags

For each shout_detail item we provide shout_details_tags as a way to quickly understand the incident. Here are the tags and details they correspond to:

Tag categories:
- Mechanical
- Rescue
- Medical
- Environmental
- Mishap
- Assistance
- FalseAlarm
- Miscellaneous
- Search
- Transport

See docs/tag_categories.md for what into each category. To obtain shout_detail_tags we read each shout_detail and manually input the tags. Alternatively one could use a LLM to classify based on tags, see docs/detail_tag_prompt.md

Reading through the shout-detials we see that LLRB also work with the follwoing organisations:
- SAS
- LLTNP
- DMMS

# Weather

We parsed weather_at_time_of_shout manually into a comma separated list of weather conditions. 

We record these changes in:

"data\codes_preprocessed_dates_time_data.csv"

Sometimes they record what the waves (the chop/swell level),temperature, light level,wind direction and strength, and visability level.

We can ask the crew to record these details at the time of the incident.

# pager_code

Pager codes initially are formed of three leading terms these are:

- 999 = someone in water,
- 333 = search, 
- 222 = urgent but no threat to life

Sometimes the main-pager-codes are followed by a sub-pager-code. We do not know what these sub-pager-codes refer to (we leave these for now).

LLRB often gets called out without need for a pager code, or with AIRWAVE. This occurs while out on training or on a previous callout.


We will ask the crew to provide more information on these. If they are not able to provide more information we will will leave the pager-codes as they are.

Given pager_code take first three letters as the code, if there are more than three letters, take the next three letters as subcode.


In [25]:
df['pager_code'].fillna("Na", inplace=True)

,pager_code,shout_details
1,333,17' Fletcher speedboat with 1 male occupant had suffered mechanical failure and was drifting just south of Ardlui. The boat and occupant were put on a long tow and taken back to their berth at Rowardennan
3,333,Reports of a small craft adrift out in front of Duckbay. After a quick search the small tender was located on the shore line. Small craft was found to be damaged and was removed from the water.
5,333,"Report of a vessel drifted from beach with no persons onboard. Caller informed Police Scotland that he was on the beach and required assistance to retrieve the vessel. LLRB launched to support, once on scene crew quickly located the caller who advised local campers were able to assist retrieve his vessel, all being well LLRB stood down and returned to base."
6,333,"Cuddy drifted ashore onto Boturich shore line with 4 on board, boat taken back to duncan mills"
7,333,"LLRB paged to assist male who had kayaked across the Loch from the A82 side but was now stranded on the west highland way after suffering damage to his inflatable , male & his kayak were taken to Luss pier to meet his family and return home , LLRB stood down returning to base at 00:48"


Identify all rows that begin with 999,333,222 and have a subcode. Extract subcode and put into subcode.

In [27]:
#First put all subcodes as None
df['subcode']=None

#First: Identify all rows that are of form 999,333,222 only. 

# Identify all rows that are of form 999,333,222 only. For the subcode put None.
df_pager_main_only = df['pager_code'].apply(lambda x: x if x in ['999','333','222'] else None)
# df_pager_main_only.value_counts()

#Now get subcodes for the rest of the rows

df['subcode'] = df['pager_code'].apply(lambda x: str(x)[3:] if x not in df_pager_main_only else None)

Extract rows that do not start with a pager code of the form 999,333,222.

In [28]:
#Identify all rows that do not start with:  999,333,222.

df_non = df[~df['pager_code'].astype(str).str.match(r'^(999|333|222)')]
# print(df_non.shape) #69
df_non['pager_code'].value_counts()

(69, 7)


pager_code
Na                                     50
other / already out                     5
On water                                2
AIRWAVE                                 2
Na on water                             2
Nil                                     1
N/A   AB messaged                       1
N/a                                     1
3032                                    1
Na already on water at safety event     1
11:30                                   1
Na already out                          1
Na just returned from previous          1
Name: count, dtype: int64

In [ ]:
#save those that begin with 999,333,222
df_main = df[df['pager_code'].astype(str).str.match(r'^(999|333|222)')]

# Once pager_code has be entered for those without (df_non), concate df_main and created df_non_entered.
# identify those pager_codes that need to be entered in for based on shout_details.
#See pager code descriptions

# save df_non to csv
# df_non.to_csv('../data/pager_codes_missing.csv', index=False)

# 'crew_on_board' and 'crew_on_shore'

Ask those recording to record the initials of the crew on board and on shore in a comma separated list. If there are no crew on board or on shore, record as "None".

In [ ]:
{
    "RB": "Ronnie Britton",
    "RO": "Rennie Oliver",
    "IG": "Iain Gollan (Goz)",
    "AM": "Ally McLeod",
    "ABS": "Andy Biddulph Snr",
    "ABJ": "Andy Biddulph Jnr",
    "GD": "Gemma Dorran",
    "PBT": "Phils Brooks-Taylor",
    "DON": "David O'Neil",
    "CC": "Craig Clancy",
    "GH": "Gerry Heaney",
    "AJM": "Angus John MacDonald",
    "CMS": "Callum MacKenzie Stevens",
    "DS": "David Stuart",
    "TR": "Thomas Rogers",
    "EM": "Euan MciIwraith",
    "PD": "Paul Dorrian",
    "KM": "Kevin McPartland",
    "JB": "Jenna Biddulph",
    "VM": "Vicki Murphy",
    "JM": "John Mason",
    "AC": "Andy Connell",
    "FN": "Franny Nicol",
    "FR": "Frank Rogers",
    "CA": "Christine Allan",
    "CS": "Clinton Salter"
    "JT": "James Thomson",
    "TAM":"Tam (Cox)",
    "GERARD","Gerard",
    "DAVY","Davy",
    "LEE","Lee",
}

#TODO! 

Remember to convert weather, details and location to lower case.

Ensure crew spaces are removed


In [ ]:
import pandas as pd 

df = pd.read_csv("..\data\codes_preprocessed_dates_time_data.csv")
# df.drop(columns=["shout_details"], inplace=True)
# df.columns

In [ ]:
#Turn values into comma separated values.
# Keep as Initials for now.

In [ ]:
df['crew_on_board'].fillna("Na", inplace=True)
df['crew_on_shore'].fillna("Na", inplace=True)

In [65]:
feats=['crew_on_board', 'crew_on_shore']
df[feats].head()
df_crew=df[feats]

In [ ]:
df_crew.value_counts()
#For crew_on_board
# seperate comma serparated values for each row. 
#count the number of times a given name appears.

#For a given name get a list of pager_codes that they have been on board for.
# Similarly get the weather for this persons callouts.